In [1]:
import numpy as np
import pyccl as ccl
import matplotlib.pyplot as plt

# Ejected Gas & Bound Gas Profiles (separate profiles)

In [120]:
class ejGas_profile(ccl.halos.profiles.profile_base.HaloProfile): #(ccl.halos.Haloprofile):
    """Creating a class for the ejected gas density profile
    where: """  # could put in the equations used

    def __init__(self, mass_def, r_ej):
        self.r_ej = r_ej  
        super(ejGas_profile, self).__init__(mass_def=mass_def)

    def _real(self, r, M, r_ej, a=1, f=1): 
        r_use = np.atleast_1d(r) # need them to be functional arrays
        M_use = np.atleast_1d(M)
        rEj_use = np.atleast_1d(r_ej)

# Returns: float or array_like: halo profile. Shape of output = `(N_M, N_r)` where `N_r` & `N_m` = the sizes of `r` and `M` respectively.
        prefix = M_use * f * (1/(a*np.sqrt(2*np.pi*rEj_use)))**3
        x = r_use[None, :] / rEj_use[:, None]
        prof = prefix[:, None] * np.exp(-(x**2)/2)

        if np.ndim(r) == 0:
            prof = np.squeeze(prof, axis=-1)
        if np.ndim(M) == 0:
            prof = np.squeeze(prof, axis=0)
                                                          
        return prof

    def _fourier(self, k, M, r_ej, a=1, f=1):
        k_use = np.atleast_1d(k)
        M_use = np.atleast_1d(M)
        rEj_use = np.atleast_1d(r_ej)

        prefix = M_use * f / a**3
        x = k_use[None, :] * rEj_use[:, None]
        prof = prefix[:, None] * np.exp(-(x**2)/2)  #prof = M_use * f * np.exp(-(1/2)*(k_use*r_ej)**2)/(a**3)

        if np.ndim(k) == 0:
            prof = np.squeeze(prof, axis=-1)
        if np.ndim(M) == 0:
            prof = np.squeeze(prof, axis=0)

        return prof
    

In [136]:
class boundGas_profile(ccl.halos.profiles.profile_base.HaloProfile): #(ccl.halos.Haloprofile):
    """Creating a class for the bound gas density profile
    where: """  # could put in the equations used

    def __init__(self, mass_def, concentration, gamma, r_s=1, GammaRange = (1.01, 10), nGamma=64, qrange=(1e-4, 1e2), nq=64):
        #, # n_q/Gamma = 2^x
                # Omega_c = 0.25, Omega_b=0.05, h=0.07, A_s=2.1e-9, n_s=0.96, Neff=0, Omega_g=0):
        # will change later, but can leave characteristic scale as ~ 1 Mpc for the moment
        
        self.r_s = r_s 
        self.gamma = gamma
        super(boundGas_profile, self).__init__(mass_def=mass_def, concentration=concentration)

        self.GammaRange = GammaRange
        self.nGamma = nGamma

        self.qrange = qrange
        self.nq = nq
        
        self._func_normQ0 = None   # General normalised profile (for q=0, over Gamma)
        self._func_normQany = None

    def _shape(self, x, gam):
        gam_use = np.atleast_1d(gam)
        return (np.log(1+x)/x)**gam_use

    def _innerInt(self, x, gam): #(a, gam=1/(gamma_use-1), b=1, c=1):
        return x**2 * self._shape(x, gam)   # eqn = (1/c) * (a**2) * (np.log(1 + a/b)/(a/b))**gam

    import scipy.integrate as integrate
    import scipy.interpolate as interpol
    from pyccl._core import UnlockInstance

    def _Vb_prefix(self, r_s=1):
        vB1 = integrate.quad(self._innerInt, 0, np.inf, args = (1/(self.gamma-1)))  # (func, a, b[, args, full_output, ...])
        vB2 = 4*np.pi*(r_s**3)*vB1[0]
        return vB2

    def _norm_interpol1(self):  # interpol1 = for q = 0
        gamma_list = np.linspace(self.GammaRange[0], self.GammaRange[1], self.nGamma) # start, stop, number of evenly spaced samples
        I0_array = np.zeros(self.nGamma)
        k=0
        for i in gamma_list:
            I0_array[k] =  integrate.quad(self._innerInt, 0, np.inf, args = 1/(i-1))[0] # _innerInt(self, x, gam=1/(gamma_use-1))
            k+=1
        func_normQ0 = interpol.interp1d(gamma_list, I0_array) # interp1d(x, y, ...
        return func_normQ0
        
    def _real(self, r, M, call_interp=True, r_s=1, scale_a=1, f=1): 
        r_use = np.atleast_1d(r) # need them to be functional arrays
        M_use = np.atleast_1d(M)
        
        R_M = self.mass_def.get_radius(cosmo, M_use, scale_a) / scale_a # halo virial radius
        c_M = self.concentration(cosmo, M_use, scale_a) # concentration-mass relation c(M)
        r_s = R_M / c_M # characteristic scale r_s

        if call_interp==False:
            vB_prefix = self._Vb_prefix(r_s)
        else:
            if self._func_normQ0 is None: # is instead of == here
                with UnlockInstance(self):
                    self._func_normQ0 = self._norm_interpol1() 
            vB_prefix = 4*np.pi*(r_s**3)*self._func_normQ0(self.gamma)  # self._Vb_prefix(gamma_use, rS_use)[0]
        prefix = M_use * f * (1/scale_a**3) * (1/vB_prefix)
# Returns: float or array_like: halo profile. Shape of the output = `(N_M, N_r)` where `N_r` & `N_m` = the sizes of `r` & `M` respectively.

        x = r_use[None, :] / r_s[:, None]
        prof = prefix[:, None] * self._shape(x, 1/(self.gamma-1)) # default par of gam = 1/(gamma_use-1) #innerInt(x)

        if np.ndim(r) == 0:
            prof = np.squeeze(prof, axis=-1)
        if np.ndim(M) == 0:
            prof = np.squeeze(prof, axis=0)
                                                          
        return prof

    def _norm_interpol2(self):  # interpol1 for q = any
        gamma_list = np.linspace(self.GammaRange[0], self.GammaRange[1], self.nGamma) # start, stop, number of evenly spaced samples
        q_array = np.geomspace(self.qrange[0], self.qrange[1], self.nq) # q = k*r_s, k will be from geomspace
        I0_array =  np.zeros((self.nGamma, self.nq))

        def integralQany(x, gam): #(a, gam=1/(gamma_use-1), b=1, c=1):
            return x * self._shape(x, gam) # for q = any (& j0(qx) = sinh(qx) = sin(qx)/(qx) => int[x^2 ... j0] = int[x ... sin(qx)]/q
        k=0
        for i in gamma_list: 
            l=0
            for j in q_array: 
                I0_array[k, l] =  integrate.quad(integralQany, 0, np.inf, args = 1/(i-1), weight = "sin", wvar=j)[0] / j
                l+=1
            k+=1
            print(f'k = {100*k/self.nGamma:.3g}% through')
        #interpol.RegularGridInterpolator() for 2D array # (points, values, ... 
        func_normQany = interpol.RegularGridInterpolator((gamma_list, np.log(q_array)), I0_array)
        return func_normQany
    
    def _fourier(self, k, M, scale_a=1, f=1):
        k_use = np.atleast_1d(k)
        M_use = np.atleast_1d(M)

        R_M = self.mass_def.get_radius(cosmo, M_use, scale_a) / scale_a # halo virial radius
        c_M = self.concentration(cosmo, M_use, scale_a) # concentration-mass relation c(M)
        r_s = R_M / c_M # characteristic scale r_s

        if self._func_normQ0 is None: # is instead of == here
            with UnlockInstance(self):
                self._func_normQ0 = self._norm_interpol1() 
        if self._func_normQany is None:
            with UnlockInstance(self):
                self._func_normQany = self._norm_interpol2()

        q_use = k_use[None, :]*r_s
        g_k = self._func_normQany((self.gamma, np.log(q_use))) / self._func_normQ0(self.gamma) # = Ib_qAny / Ib_q0

        prefix = M_use * f / a**3
        prof = prefix[:, None] * g_k[None,:] 

        if np.ndim(k) == 0:
            prof = np.squeeze(prof, axis=-1)
        if np.ndim(M) == 0:
            prof = np.squeeze(prof, axis=0)

        return prof
    

# New Combined Class

Add in equations: $\rho(r)\ = f_b\ \rho_{bound}(r)\ + (1 - f_b)\rho_{ejected}(r)\ $

In [60]:
combinedGas_profile._cd_fourier

<function __main__.combinedGas_profile._cd_fourier(self, k, M, r_ej, f_bd=1, call_interp=True, r_s=1, scale_a=1)>

In [166]:
class combinedGas_profile(boundGas_profile, ejGas_profile): # ccl.halos.profiles.profile_base.HaloProfile, 
    """Text
    """

    def __init__(self, mass_def, concentration, gamma, r_ej, r_s=1, GammaRange = (1.01, 10), nGamma=64, qrange=(1e-4, 1e2), nq=64):
        
        self.r_ej = r_ej  
        self.r_s = r_s 
        self.gamma = gamma
        # super().__init__(mass_def=mass_def, concentration=concentration)
        
       # super(combinedGas_profile, self).__init__(mass_def=mass_def, concentration=concentration)
      #  ejGas_profile.__init__(self, mass_def=mass_def, r_ej=r_ej)
        boundGas_profile.__init__(self, mass_def=mass_def, concentration=concentration, gamma = gamma)
        A.__init__(self, x=x, y=y)

        self.GammaRange = GammaRange
        self.nGamma = nGamma
        self.qrange = qrange
        self.nq = nq
        
        self._func_normQ0 = None   # General normalised bound profile (for q=0, over Gamma)
        self._func_normQany = None

    def _real(self, r, M, r_ej, f_bd=1, call_interp=True, r_s=1, scale_a=1):
        f_ej = 1 - f_bd
        prof_ej = ejGas_profile._real(self, r, M, r_ej, scale_a, f_ej)
        prof_bd = boundGas_profile._real(self, r, M, call_interp, r_s, scale_a, f_bd)
        profile = prof_ej + prof_bd
        return profile

    def _fourier(self, k, M, r_ej, f_bd = 1, call_interp=True, r_s=1, scale_a=1):
        f_ej = 1 - f_b
        prof_ej = ejGas_profile._fourier(self, k, M, r_ej, scale_a, f_ej)
        prof_bd = boundGas_profile._fourier(self, k, M, scale_a, f_bd)
        profile = prof_ej + prof_bd
        return profile

In [154]:
class combinedGas_profile(boundGas_profile, ejGas_profile): # ccl.halos.profiles.profile_base.HaloProfile, 
    """Text
    """

    def __init__(self, mass_def, gamma, concentration, r_ej, r_s=1, GammaRange = (1.01, 10), nGamma=64, qrange=(1e-4, 1e2), nq=64):
        
        self.r_ej = r_ej  
        self.r_s = r_s 
        self.gamma = gamma
        super().__init__(mass_def=mass_def, concentration=concentration)
        
        # super(combinedGas_profile, self).__init__(mass_def=mass_def, concentration=concentration)
        #boundGas_profile.__init__(self)

        self.GammaRange = GammaRange
        self.nGamma = nGamma
        self.qrange = qrange
        self.nq = nq
        
        self._func_normQ0 = None   # General normalised bound profile (for q=0, over Gamma)
        self._func_normQany = None

    def _cd_real(self, r, M, r_ej, f_bd=1, call_interp=True, r_s=1, scale_a=1):
        f_ej = 1 - f_bd
        prof_ej = self._ej_real(self, r, M, r_ej, scale_a, f_ej)
        prof_bd = self._bd_real(self, r, M, call_interp, r_s, scale_a, f_bd)
        profile = prof_ej + prof_bd
        return profile

    def _cd_fourier(self, k, M, r_ej, f_bd = 1, call_interp=True, r_s=1, scale_a=1):
        f_ej = 1 - f_b
        prof_ej = self._ej_fourier(self, k, M, r_ej, scale_a, f_ej)
        prof_bd = self._bd_fourier(self, k, M, scale_a, f_bd)
        profile = prof_ej + prof_bd
        return profile

In [100]:
hmd_200c = ccl.halos.MassDef200c # use a mass definition with Delta = 200
cM = ccl.halos.ConcentrationDuffy08(mass_def=hmd_200c)
cosmo = ccl.Cosmology(Omega_c = 0.25, Omega_b=0.05, h=0.07, A_s=2.1e-9, n_s=0.96, Neff=0, Omega_g=0)

trial_r = np.geomspace(1E-2,1E8,100000) # geomspace is like linspace, but evenly spaces on a logscale
trial_M = np.array([5E12,2E13, 1E14])
trial_gamma = 1.1  # working with z = 0 (today) ; therefore a = 1 [default]
trial_rEj = 100
trial_rS = 1       

In [168]:
trial_profile = combinedGas_profile(hmd_200c, cM, trial_gamma, trial_rEj)

TypeError: ejGas_profile.__init__() got an unexpected keyword argument 'concentration'

In [72]:
boundGas_profile.__init__

<function __main__.boundGas_profile.__init__(self, mass_def, gamma, concentration, r_s=1, GammaRange=(1.01, 10), nGamma=64, qrange=(0.0001, 100.0), nq=64)>

In [152]:
ejGas_profile.__init__

<function __main__.ejGas_profile.__init__(self, mass_def, r_ej)>